# word2vec 실습
reference: [딥러닝을 이용한 자연어 처리 입문](wikidocs.net/50739)

## 훈련 데이터 전처리
  
xml 파일에서 자연어를 얻어내기 위해 전처리를 해준다. 자연어는 `<content>`와 `</content>`에 위치함을 유의하면서 처리한다.

In [1]:
import re
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

In [2]:
# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)
parse_text = '\n'.join(target_text.xpath('//content/text()'))

In [3]:
# (Audio), (Laughter) 등의 배경음 부분 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

In [4]:
from nltk import download
download('punkt')
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text=sent_tokenize(content_text)

[nltk_data] Downloading package punkt to C:\Users\SHIN-
[nltk_data]     DESKTOP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)

In [6]:
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = []
result = [word_tokenize(sentence) for sentence in normalized_text]

In [7]:
print('총 샘플의 개수 : {}'.format(len(result)))
for line in result[:5]:
    print(line)

총 샘플의 개수 : 273424
['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']
['consider', 'facit']
['i', 'm', 'actually', 'old', 'enough', 'to', 'remember', 'them']


## word2vec 훈련

아래는 word2vec의 하이퍼 파라미터 값에 대한 설명이다.

- size : 임베딩된 벡터의 차원
- window: 컨텍스트 윈도우 크기
- min_count: 단어 최소 빈도 수 제한. 빈도수가 너무 적으면 학습하지 않는다.
- workers: 학습을 위한 프로세스 수
- sg: 0은 CBOW, 1은 skip-gram 방식

In [8]:
# 학습 진행
from gensim.models import Word2Vec
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

학습된 결과로 man과 가장 유사한 단어는 무엇이 있을지 살펴보자

In [9]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.862735390663147), ('guy', 0.7980318069458008), ('lady', 0.7647310495376587), ('girl', 0.7404158115386963), ('boy', 0.7392442226409912), ('gentleman', 0.72615647315979), ('soldier', 0.7101109027862549), ('kid', 0.6777242422103882), ('friend', 0.6699459552764893), ('david', 0.6625057458877563)]
